In [ ]:
import json
import os
import math
import librosa

from google.colab import drive
drive.mount('/content/drive')

#DATASET_PATH = '/content/drive/MyDrive/8th/Thesis/Emotion Music/'
DATASET_PATH = '/content/drive/MyDrive/Thesis/Emotion Music/'
#DATASET_PATH = '/content/drive/MyDrive/Thesis/Data/genres_original/'


Mounted at /content/drive


In [ ]:
#JSON_PATH = "/content/drive/MyDrive/Thesis/Emotion Music/data_new_10.json"
#JSON_PATH = "/content/drive/MyDrive/Colab Notebooks/data_genres_New_10.json"
JSON_PATH = "/content/drive/MyDrive/Colab Notebooks/data_emotion_New_4.json"
#SAMPLE_RATE = 22050
SAMPLE_RATE = 44100
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

In [ ]:
def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.

        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all genre sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we're processing a genre sub-folder level
        if dirpath is not dataset_path:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            #print("\nProcessing: {}".format(semantic_label))

            # process all audio files in genre sub-dir
            for f in filenames:

		#  load audio file
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # process all segments of audio file
                for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # extract mfcc
                    mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T                                     

                    # store only mfcc feature with expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        #print("{}, segment:{}".format(file_path, d+1))

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

def main():
    save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)       
if __name__ == "__main__":
    main()

In [ ]:
import os

filepaths = [os.path.join(DATASET_PATH, f) for f in os.listdir(DATASET_PATH)]
for (dirpath, dirnames, filenames) in os.walk(DATASET_PATH):
    for f in filenames:
        print('FILE :', os.path.join(dirpath, f))
    for d in dirnames:
        print('DIRECTORY :', os.path.join(dirpath, d))

DIRECTORY : /content/drive/MyDrive/Thesis/Emotion Music/sad
DIRECTORY : /content/drive/MyDrive/Thesis/Emotion Music/relax
DIRECTORY : /content/drive/MyDrive/Thesis/Emotion Music/angry
DIRECTORY : /content/drive/MyDrive/Thesis/Emotion Music/happy
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00023.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00001.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00002.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00003.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00004.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00005.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00006.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00007.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00008.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00009.wav
FILE : /content/drive/MyDrive/Thesis/Emotion Music/sad/sad.00010

In [ ]:
    import numpy as np
    
    with open(JSON_PATH, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    X.shape
    #y.shape
    #X
    #y

(3098, 259, 13)